In [71]:
# Anwendung von NLP-Techniken

# Import der erforderlichen Bibliotheken
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
# Laden der Daten (csv.-Datei). Der Pfad muss entsprechend angepasst werden
df = pd.read_csv("C:\\Users\\TC\\Desktop\\rows.csv")
df.shape

# Es erscheint eine Warnung da gemischte Datentypen in den Spalten vorhanden sind, diese kann hier aber ignoriert werden
# Alternativ entsprechend den Vorschlägen behandeln, was wir hier aber nicht tun

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\TC\AppData\Local\Temp\ipykernel_26528\2296427209.py:7: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\\Users\\TC\\Desktop\\rows.csv")


(1282355, 18)

In [72]:
# Der Datensatz besteht aus über ine Millionen Zeilen und 18 Spalten
# Anzeige der ersten 3 Zeilen der Daten und Transponieren des Ergebnisses
df.head(3).T
#print(df)

,0,1,2
Date received,05/10/2019,05/10/2019,05/10/2019
Product,Checking or savings account,Checking or savings account,Debt collection
Sub-product,Checking account,Other banking product or service,Payday loan debt
Issue,Managing an account,Managing an account,Communication tactics
Sub-issue,Problem using a debit or ATM card,Deposits and withdrawals,Frequent or repeated calls
Consumer complaint narrative,NaN,NaN,NaN
Company public response,NaN,NaN,NaN
Company,NAVY FEDERAL CREDIT UNION,BOEING EMPLOYEES CREDIT UNION,CURO Intermediate Holdings
State,FL,WA,TX
ZIP code,328XX,98204,751XX


In [73]:
# Entfernung irrelevanter Daten wie Adressen oder Datum, um den Datensatz zu verkleinern
# Der kleinere Datensatz wird neu abgespeichert
df1 = df[['Product', 'Consumer complaint narrative']].copy()

# Entfernen der Datensätze, welche keine Beschwerde enthalten, also unter "Consumer complaint narrative" ein "NaN"
df1 = df1[pd.notnull(df1['Consumer complaint narrative'])]

# Einfachere Benennung der zwei Spalten einfügen
df1.columns = ['Product', 'Complaint'] 

# Ausgabe er Anzahl übriger Beschwerden zur weiteren Verarbeitung
df1.shape

# Optional Visualisierung der Daten
# print(df)


(383564, 2)

In [74]:
# Der kleinere Datensatz besteht aus über 383000 Beschwerden und 2 Spalten 
# Optional zur Visualisieurng der Daten
df1.head(5)

,Product,Complaint
29904,"Credit reporting, credit repair services, or o...",The Summer of XX/XX/2018 I was denied a mortga...
30629,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30735,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30795,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30807,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...


In [75]:
# Anzeige der Unterschiedlichen Kategorien (Product) aus dem Datensatz
# pd.DataFrame(df.Product.unique()).values


In [76]:
# Anzeige der Anzahl der Kategorien (Product) aus dem Datensatz
count_unique_products = df.Product.nunique()
print(count_unique_products)

18


In [77]:
# Hier die Anzeige er Zeilen mit den Kategorien
df1['Product'].value_counts()

Product
Credit reporting, credit repair services, or other personal consumer reports    92378
Debt collection                                                                 86710
Mortgage                                                                        52987
Credit reporting                                                                31588
Student loan                                                                    21810
Credit card or prepaid card                                                     21379
Credit card                                                                     18838
Bank account or service                                                         14885
Checking or savings account                                                     12881
Consumer Loan                                                                    9474
Vehicle loan or lease                                                            5745
Money transfer, virtual currency, or money ser

In [100]:
# Da der Datensatz noch sehr groß ist und mein System an seine Grenzen in Bezug auf die Rechenleistung kommt 
# wird nun lediglich die Kategorie "Student loan" weiter verarbeitet
# Beispielhafte Darstellung der ersten 5 Einträge
df2 = df1[df1['Product'] == 'Student loan']
df2.head(5)

,Product,Complaint
35319,Student loan,I met the requirements for the ability to bene...
38191,Student loan,Navient refuses to issue a paid in full letter...
38829,Student loan,XX/XX//2019 - I sent a letter to the Dept of E...
38999,Student loan,"For years, Navient ( previously Sallie Mae ) e..."
39223,Student loan,I am the co signer on my daughter 's student l...


In [101]:
# Importieren der erforderlichen Bibliothek für das Stemming
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

# Erstellen des Stemmers
stemmer = SnowballStemmer("english")

# Definition einer Funktion, um das Stemming auf einen Satz anzuwenden
def stem_sentence(sentence):
    words = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Funktion auf die Spalte 'Complaint' anwenden
df2.loc['Complaint'] = df2['Complaint'].apply(stem_sentence)

df2.head(5)
# 

C:\Users\TC\AppData\Local\Temp\ipykernel_26528\3273816499.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc['Complaint'] = df2['Complaint'].apply(stem_sentence)


,Product,Complaint
35319,Student loan,I met the requirements for the ability to bene...
38191,Student loan,Navient refuses to issue a paid in full letter...
38829,Student loan,XX/XX//2019 - I sent a letter to the Dept of E...
38999,Student loan,"For years, Navient ( previously Sallie Mae ) e..."
39223,Student loan,I am the co signer on my daughter 's student l...


In [80]:
# Da der Datensatz noch sehr groß ist und mein System an seine Grenzen in Bezug auf die Rechenleistung kommt 
# wird eine Stichprobe von 10000 Einträgen gezogen, welche weiter verarbeitet wird
# Falls genügend Rechenkapazität zur verfügung steht kann dieser Schritt übersprungen werden
# und der Code muss entsprechend auf den Ursprungsdatensatz angepasst werden
# df2 = df1.sample(10000, random_state=1).copy()

In [81]:
# Da es ähnliche Kategorien gibt, werden diese zusammengefasst und entsprechend umbenannt
# df2.replace({'Product': 
  #            {'Credit reporting, credit repair services, or other personal consumer reports': 
  #            'Credit reporting, repair, or other', 
  #            'Credit reporting': 'Credit reporting, repair, or other',
  #           'Credit card': 'Credit card or prepaid card',
  #           'Prepaid card': 'Credit card or prepaid card',
  #           'Payday loan': 'Payday loan, title loan, or personal loan',
  #           'Money transfer': 'Money transfer, virtual currency, or money service',
  #           'Virtual currency': 'Money transfer, virtual currency, or money service'}}, 
  #          inplace= True)

In [82]:
# Anzeige der nun vorhandenen Kategorien
# pd.DataFrame(df2.Product.unique())

In [83]:
# Erstellen einer neuen Spalte, um die Kategorien zu kodieren
#df2['category_id'] = df2['Product'].factorize()[0]
#category_id_df = df2[['Product', 'category_id']].drop_duplicates()


# Ein Wörterbuch wird erstellt
#category_to_id = dict(category_id_df.values)
#id_to_category = dict(category_id_df[['category_id', 'Product']].values)

# Anzeigen des neuen Datenframes mittels beispielhafter Ausgabe der ersten 5 Zeilen 
#df2.head()

In [88]:
# BOW-Vektorisierung
# Import der Klasse CountVectorizer für BOW
from sklearn.feature_extraction.text import CountVectorizer

# Erstellen des CountVectorizer-Objekts
# Eingrenzung auf 100 Merkmale
# Berücksichtigung von Unigramme und Bigramme (ngram_range)
# Erstellung einer Liste mit englische und aufgrund der Daten definierten Stoppwörtern

stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['xx','xxx','xxxx']
stopwords.extend(newStopWords)

# Entfernung der Stoppwörter aus der Liste für englische Stoppwörter (stop_words)

bow = CountVectorizer(max_features=100, min_df=1000, ngram_range=(1, 2), stop_words= stopwords)

# Umwandlung der Beschwerden in einen BoW-Vektor
bow_features = bow.fit_transform(df2.Complaint).toarray()

print(df2.columns)

#labels = df2.category_id

#Ausagbe der Ergebnisse
print("Jede der %d Beschwerden wird dargestellt durch %d Merkmale" %(bow_features.shape))

Index(['Product', 'Complaint'], dtype='object')
Jede der 21810 Beschwerden wird dargestellt durch 100 Merkmale


In [90]:
# TFIDF-Vektorisierung
# Import des TFIDF Vektorizers


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Entfernung von Wörtern, welche in weniger als 1000 Dokumenten vorkommen (min_df)
# Berücksichtigung von Unigramme und Bigramme (ngram_range)
# Entfernung der Stoppwörter aus der Liste für englische Stoppwörter (stop_words) und Datensatzspetifische Stoppwörter
# Eingrenzung auf 100 Merkmale

tfidf = TfidfVectorizer(max_features=100,sublinear_tf=True, min_df=1000,
                        ngram_range=(1, 2), 
                        stop_words=stopwords)

# Jede Beschwerde wird in einen TFIDF-Vektor umgewandelt
tfidf_features = tfidf.fit_transform(df2.Complaint).toarray()

print(df2.columns)

#labels = df2.category_id

print("Jede der %d Beschwerden wird dargestellt durch %d Merkmale" %(tfidf_features.shape))

Index(['Product', 'Complaint'], dtype='object')
Jede der 21810 Beschwerden wird dargestellt durch 100 Merkmale


In [91]:
# Wir sehen nun die Anzahl der Merkmale ist identisch, weil beide Methoden das Vokabular 
# aus den gleichen Eingabedaten erstellen. Beide Methoden betrachten jedes eindeutige Wort 
# in den Daten als ein Merkmal.
# Die Unterschiede zwischen TF-IDF und BoW liegen in der Art und Weise, wie sie die Werte für diese Merkmale berechnen

In [92]:
# Ausgabe der ersten 10 Merkmale für das erste Dokument jeweils für BOW und TF-IDF
print("BoW Merkmale für das erste Dokument: ", bow_features[0][:10])
print("TF-IDF Merkmale für das erste Dokument: ", tfidf_features[0][:10])

BoW Merkmale für das erste Dokument:  [0 0 0 0 0 0 0 1 0 0]
TF-IDF Merkmale für das erste Dokument:  [0.         0.         0.         0.         0.         0.
 0.         0.24086021 0.         0.        ]


In [93]:
# Nun erkennt man, dass in großen Textsammlungen viele Nullvektoren bei BOW und TF-IDF entstehen, weshalb nochmals oben
# mittels min_df auf noch weniger Merkmale reduziert werden musste, was aber immer noch zu vielen Nullvektoren führt
# Auch zu sehen ist in dem Beispiel, dass die Werte in den BoW-Vektoren höher sind da nur die Häufigkeit jedes Wortes 
# in einem Dokument berücksichtigt wird
# während die Werte in den TF-IDF-Vektoren kleiner sind, da die Wortzählungen nach ihrer Bedeutung in den Dokumenten
# gewichtet werden

In [94]:
# Anzeige der 100 Merkmale
print("BoW Merkmale: ", bow.get_feature_names_out())
print("TF-IDF Merkmale: ", tfidf.get_feature_names_out())

BoW Merkmale:  ['00' 'abl' 'account' 'also' 'amount' 'ani' 'appli' 'ask' 'back' 'balanc'
 'becaus' 'bill' 'call' 'chang' 'check' 'collect' 'compani' 'consolid'
 'contact' 'continu' 'could' 'credit' 'current' 'date' 'day' 'debt'
 'defer' 'document' 'due' 'educ' 'email' 'even' 'everi' 'feder' 'financi'
 'first' 'forbear' 'get' 'go' 'help' 'incom' 'inform' 'interest' 'issu'
 'know' 'late' 'letter' 'like' 'loan' 'made' 'make' 'make payment' 'money'
 'month' 'month payment' 'navient' 'need' 'never' 'number' 'one' 'onli'
 'option' 'owe' 'paid' 'pay' 'payment' 'phone' 'plan' 'privat' 'process'
 'program' 'provid' 'qualifi' 'rate' 'receiv' 'repay' 'report' 'repres'
 'request' 'said' 'say' 'school' 'sent' 'servic' 'sign' 'sinc' 'start'
 'state' 'still' 'student' 'student loan' 'take' 'time' 'told' 'tri'
 'want' 'whi' 'work' 'would' 'year']
TF-IDF Merkmale:  ['00' 'abl' 'account' 'also' 'amount' 'ani' 'appli' 'ask' 'back' 'balanc'
 'becaus' 'bill' 'call' 'chang' 'check' 'collect' 'compani' 'cons

In [102]:
# Die Themenanalyse wird mit den TF-IDF-Vektoren durchgeführt, da diese in der Regel bessere Ergebnisse als BOW liefert
# Berechnen der durchschnittlichen TF-IDF-Werte für jedes Wort
avg_tfidf = np.mean(tfidf_features, axis=0)

# Erstellen eines DataFrames mit den Wörtern und ihren durchschnittlichen TF-IDF-Werten
df_tfidf = pd.DataFrame({'word': tfidf.get_feature_names_out(), 'avg_tfidf': avg_tfidf})

# Sortieren des DataFrames in absteigender Reihenfolge der durchschnittlichen TF-IDF-Werte
df_tfidf = df_tfidf.sort_values('avg_tfidf', ascending=False)

# Ausgabe der Wörter und ihrer durchschnittlichen TF-IDF-Werte
print(df_tfidf)

       word  avg_tfidf
48     loan   0.137340
65  payment   0.116937
55  navient   0.084669
64      pay   0.081467
53    month   0.078915
..      ...        ...
77   repres   0.027404
14    check   0.026931
35    first   0.026683
11     bill   0.026607
72  qualifi   0.025350

[100 rows x 2 columns]


In [97]:
# Extraktion der 5 häufigsten Themen jeder Kategorie mittels sklearn LDA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation

# Erstellung des LDA-Modells
lda_model=LatentDirichletAllocation(n_components=5,learning_method=
'online',random_state=42,max_iter=1)

# Anpassung des LDA-Models an die Merkmale
lda_top=lda_model.fit_transform(tfidf_features)

# Ausgabe der Themenverteilung für jede Beschwerde in Prozent
for i,topic in enumerate(lda_top[0]):
  print("Thema ",i,": ",topic*100,"%")
feature_names = tfidf.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
  print("\n==> Thema %d:" %(topic_idx))
  print("  * Schlüsselwörter: ", " ".join([feature_names[i] for i in topic.argsort()[:-N - 1:-1]]))

Thema  0 :  5.316834237188696 %
Thema  1 :  5.293745237440426 %
Thema  2 :  5.354632663557727 %
Thema  3 :  78.72856980692465 %
Thema  4 :  5.306218054888494 %

==> Thema 0:
  * Schlüsselwörter:  payment call account month loan

==> Thema 1:
  * Schlüsselwörter:  report credit late loan account

==> Thema 2:
  * Schlüsselwörter:  loan pay payment school 00

==> Thema 3:
  * Schlüsselwörter:  loan collect call letter inform

==> Thema 4:
  * Schlüsselwörter:  loan payment repay plan incom


In [103]:
# Ausgabe der beispielhaften Themenverteilung in Prozent der ersten 10 Beschwerden
for i, topic_dist in enumerate(lda_top[:10]):
  print("\n==> Beschwerde %d:" %(i))
  print("  * Themenverteilung: ", topic_dist)


==> Beschwerde 0:
  * Themenverteilung:  [0.05316834 0.05293745 0.05354633 0.7872857  0.05306218]

==> Beschwerde 1:
  * Themenverteilung:  [0.8045643  0.0489093  0.04881546 0.0491925  0.04851844]

==> Beschwerde 2:
  * Themenverteilung:  [0.0426426  0.0446909  0.04231398 0.8281303  0.04222222]

==> Beschwerde 3:
  * Themenverteilung:  [0.04978296 0.04987832 0.05005855 0.0498509  0.80042928]

==> Beschwerde 4:
  * Themenverteilung:  [0.03352709 0.03364514 0.86630782 0.0332987  0.03322126]

==> Beschwerde 5:
  * Themenverteilung:  [0.0351619  0.03484355 0.03519312 0.85893728 0.03586416]

==> Beschwerde 6:
  * Themenverteilung:  [0.04108031 0.04070662 0.0410367  0.83432505 0.04285132]

==> Beschwerde 7:
  * Themenverteilung:  [0.03548598 0.03517975 0.85857543 0.03529031 0.03546853]

==> Beschwerde 8:
  * Themenverteilung:  [0.0332184  0.51097288 0.03306223 0.38982165 0.03292484]

==> Beschwerde 9:
  * Themenverteilung:  [0.03517969 0.03498705 0.85972244 0.03479758 0.03531323]


In [106]:
# Extraktion der 5 häufigsten Themen jeder Kategorie mittels sklearn LSA

# Erstellen des LSA-Modells
lsa_model = TruncatedSVD(n_components=5, random_state=42)

# Anpassung des LSA-Modells an die Merkmale
lsa_top = lsa_model.fit_transform(tfidf_features)

# Ausgabe der Themenverteilung für jede Beschwerde
for i, topic in enumerate(lsa_top[:5]):
    print("\n==> Thema %d:" %(i))
    print("  * Themenverteilung: ", topic)


==> Thema 0:
  * Themenverteilung:  [ 0.20224455  0.08422113 -0.05759419 -0.17140524 -0.02929152]

==> Thema 1:
  * Themenverteilung:  [ 0.33271317  0.0895126  -0.03293767  0.11379042 -0.12011335]

==> Thema 2:
  * Themenverteilung:  [0.3468383  0.46982186 0.0979874  0.12090659 0.03421351]

==> Thema 3:
  * Themenverteilung:  [ 0.36521127 -0.10412733 -0.01524876 -0.03755352  0.24534818]

==> Thema 4:
  * Themenverteilung:  [ 0.5359543   0.0088847   0.00944483 -0.18231323 -0.13377392]


In [107]:
# Ausgabe der Schlüsselwörter für jedes Thema für LSA
feature_names = tfidf.get_feature_names_out()
for topic_idx, topic in enumerate(lsa_model.components_[:5]):
    print("\n==> Thema %d:" %(topic_idx))
    print("  * Schlüsselwörter: ", " ".join([feature_names[i] for i in topic.argsort()[:-N - 1:-1]]))


==> Thema 0:
  * Schlüsselwörter:  loan payment pay navient 00

==> Thema 1:
  * Schlüsselwörter:  report credit account late inform

==> Thema 2:
  * Schlüsselwörter:  report credit student loan student loan

==> Thema 3:
  * Schlüsselwörter:  payment late account month interest

==> Thema 4:
  * Schlüsselwörter:  servic repay student loan student request
